<a href="https://colab.research.google.com/github/ro-hit81/INTERNSHIP/blob/ro-hit81-patch-1/deep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAND USE LAND CLASSIFICATION MAP FOR UBON


![Methodology Flowchart](https://drive.google.com/uc?id=189WJOrRBKdHAqgOU_tYWVwLfsWp26fwv)

## Authenticating to Google colab and Google CLOUD.
---


In [0]:
from google.colab import auth

auth.authenticate_user()

## Authenticating to Earth Engine and Initializing

In [0]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/wAEiY8tz3iB0gHt9_w-940zw4wYUNyD3VO2Y1uZanJfEYurczidUJpA

Successfully saved authorization token.


### Importing Tensorflow library & Noting its version

In [0]:
import tensorflow as tf
tf.enable_eager_execution()
print(tf.__version__)

1.15.0


## Preparing Sentinel imagery for Ubon & Visualizing using Folium.

In [0]:
import folium
bands = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12', 'QA60']
bound = ee.FeatureCollection('users/rhtkhati/Ubon_Boundary')
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0)
    mask = mask.bitwiseAnd(cirrusBitMask).eq(0)

    return image.updateMask(mask).divide(10000)
collection = (ee.ImageCollection("COPERNICUS/S2")
              .select(bands)
              .filter(ee.Filter.date('2019-01-01', '2019-03-31'))
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 1))
              .map(maskS2clouds)
             )
def add_ee_layer(self, eeImageObject, visParams, name):
  mapID = ee.Image(eeImageObject).getMapId(visParams)
  folium.TileLayer(
    tiles = mapID['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer
image = collection.sort('system:index', opt_ascending=False).mosaic()
image = image.clip(bound)
visParams1 = {'bands': ["B4","B3","B2"],
            'max': 0.4,
            'min': 0
                }
visParams2 = {'bands': ["B11","B8","B3"],
            'max': 0.4,
            'min': 0
                }
visParams3 = {'bands': ["B8","B4","B3"],
            'max': 0.4,
            'min': 0,
                }              

myMap = folium.Map(location = [15.2448, 104.8473], zoom_start = 10)
myMap.add_ee_layer(image, visParams1, 'True_Color')
myMap.add_ee_layer(image, visParams3, 'False_Color')
myMap.add_ee_layer(image, visParams2, 'Natural_Infrared')
myMap.add_child(folium.LayerControl())
myMap

## Importing base data from EE & Preparing testing & training dataset from base data.


In [0]:
labels = ee.FeatureCollection('users/rhtkhati/ubon_lulcpoint')
label = 'lulcid'
sample = image.sampleRegions(
    collection = labels, properties = [label], scale = 10).randomColumn()
training = sample.filter(ee.Filter.lt('random', 0.7))
testing = sample.filter(ee.Filter.gte('random', 0.7))
from pprint import pprint
pprint({'training': training.first().getInfo()})
pprint({'testing': testing.first().getInfo()})

{'training': {'geometry': None,
              'id': '00000000000000000000_0',
              'properties': {'B11': 0.23409999907016754,
                             'B12': 0.12110000103712082,
                             'B2': 0.1136000007390976,
                             'B3': 0.11020000278949738,
                             'B4': 0.10480000078678131,
                             'B8': 0.28439998626708984,
                             'QA60': 0,
                             'lulcid': 1,
                             'random': 0.2700527436961804},
              'type': 'Feature'}}
{'testing': {'geometry': None,
             'id': '00000000000000000006_0',
             'properties': {'B11': 0.2069000005722046,
                            'B12': 0.09600000083446503,
                            'B2': 0.10159999877214432,
                            'B3': 0.09560000151395798,
                            'B4': 0.07999999821186066,
                            'B8': 0.2703999876976013,
   

## Exporting datasets to Cloud storage bucket in TFRecord for TensorFlow model to access them.

In [0]:
outputBucket = 'gic-rohit12.appspot.com'
print('Found Cloud Storage bucket.' if tf.gfile.Exists('gs://' + outputBucket) 
    else 'Output Cloud Storage bucket does not exist.')


Found Cloud Storage bucket.


In [0]:
trainFilePrefix = 'Training_demo'
testFilePrefix = 'Testing_demo'
featureNames = list(bands)
featureNames.append(label)

trainingTask = ee.batch.Export.table.toCloudStorage(
  collection = training,
  description ='Training Export',
  fileNamePrefix = trainFilePrefix,
  bucket = outputBucket,
  fileFormat = 'TFRecord',
  selectors = featureNames)

testingTask = ee.batch.Export.table.toCloudStorage(
  collection = testing,
  description = 'Testing Export',
  fileNamePrefix = testFilePrefix,
  bucket = outputBucket,
  fileFormat = 'TFRecord',
  selectors = featureNames)

In [0]:
trainingTask.start()
testingTask.start()

In [0]:
print(ee.batch.Task.list())
import time 
while trainingTask.active():
  print('Polling for task (id: {}).'.format(trainingTask.id))
  time.sleep(30)
print('Done with training export.')

[<Task EXPORT_FEATURES: Testing Export (READY)>, <Task EXPORT_FEATURES: Training Export (READY)>, <Task INGEST: Asset ingestion: users/rhtkhati/Classified_pixel_demo (FAILED)>, <Task EXPORT_IMAGE: Image Export (COMPLETED)>, <Task EXPORT_FEATURES: Testing Export (COMPLETED)>, <Task EXPORT_FEATURES: Training Export (COMPLETED)>, <Task EXPORT_IMAGE: Image Export (COMPLETED)>, <Task EXPORT_FEATURES: Testing Export (COMPLETED)>, <Task EXPORT_FEATURES: Training Export (COMPLETED)>, <Task EXPORT_IMAGE: Image Export (COMPLETED)>, <Task EXPORT_FEATURES: Testing Export (COMPLETED)>, <Task EXPORT_FEATURES: Training Export (COMPLETED)>]
Polling for task (id: 4HOL2QI27WFQ4B4J4UHBBWBV).
Done with training export.


In [0]:
fileNameSuffix = '.tfrecord.gz'
trainFilePath = 'gs://' + outputBucket + '/' + trainFilePrefix + fileNameSuffix
testFilePath = 'gs://' + outputBucket + '/' + testFilePrefix + fileNameSuffix

print('Found training file.' if tf.gfile.Exists(trainFilePath) 
    else 'No training file found.')
print('Found testing file.' if tf.gfile.Exists(testFilePath) 
    else 'No testing file found.')

Found training file.
Found testing file.


In [0]:
imageFilePrefix = 'Image_pixel_demo_'
imageExportFormatOptions = {
  'patchDimensions': [256, 256],
  'maxFileSize': 104857600,
  'compressed': True
}
exportRegion = ee.Geometry.Rectangle([104.78310511361235, 15.075894756514106, 105.19646570931548, 15.314444748491846])
imageTask = ee.batch.Export.image.toCloudStorage(
  image=image,
  description='Image Export',
  fileNamePrefix=imageFilePrefix,
  bucket=outputBucket,
  scale=10,
  fileFormat='TFRecord',
  region=exportRegion.toGeoJSON()['coordinates'],
  formatOptions=imageExportFormatOptions,
)

In [0]:
imageTask.start()

In [0]:
while imageTask.active():
  print('Polling for task (id: {}).'.format(imageTask.id))
  time.sleep(30)
print('Done with image export.')

Polling for task (id: E46UP53OVOHNO7XA5OGYC64F).
Polling for task (id: E46UP53OVOHNO7XA5OGYC64F).
Polling for task (id: E46UP53OVOHNO7XA5OGYC64F).
Polling for task (id: E46UP53OVOHNO7XA5OGYC64F).
Polling for task (id: E46UP53OVOHNO7XA5OGYC64F).
Polling for task (id: E46UP53OVOHNO7XA5OGYC64F).
Polling for task (id: E46UP53OVOHNO7XA5OGYC64F).
Polling for task (id: E46UP53OVOHNO7XA5OGYC64F).
Polling for task (id: E46UP53OVOHNO7XA5OGYC64F).
Polling for task (id: E46UP53OVOHNO7XA5OGYC64F).
Polling for task (id: E46UP53OVOHNO7XA5OGYC64F).
Polling for task (id: E46UP53OVOHNO7XA5OGYC64F).
Polling for task (id: E46UP53OVOHNO7XA5OGYC64F).
Polling for task (id: E46UP53OVOHNO7XA5OGYC64F).
Polling for task (id: E46UP53OVOHNO7XA5OGYC64F).
Polling for task (id: E46UP53OVOHNO7XA5OGYC64F).
Polling for task (id: E46UP53OVOHNO7XA5OGYC64F).
Polling for task (id: E46UP53OVOHNO7XA5OGYC64F).
Polling for task (id: E46UP53OVOHNO7XA5OGYC64F).
Polling for task (id: E46UP53OVOHNO7XA5OGYC64F).
Polling for task (id

## Preprocessing of dataset in suitable format for input to model.


In [0]:
trainDataset = tf.data.TFRecordDataset(trainFilePath, compression_type='GZIP')
print(iter(trainDataset).next())

tf.Tensor(b'\n\x88\x01\n\x0e\n\x02B2\x12\x08\x12\x06\n\x04_\x98\x0c>\n\x0e\n\x02B3\x12\x08\x12\x06\n\x04KY\x06>\n\x0e\n\x02B4\x12\x08\x12\x06\n\x04?5\x1e>\n\x0e\n\x02B8\x12\x08\x12\x06\n\x04\x95\xd4\x89>\n\x0f\n\x03B11\x12\x08\x12\x06\n\x04O@\xb3>\n\x0f\n\x03B12\x12\x08\x12\x06\n\x04q=J>\n\x10\n\x04QA60\x12\x08\x12\x06\n\x04\x00\x00\x00\x00\n\x12\n\x06lulcid\x12\x08\x12\x06\n\x04\x00\x00\x80?', shape=(), dtype=string)


In [0]:
columns = [
  tf.io.FixedLenFeature(shape=[1], dtype=tf.float32) for k in featureNames
]
featuresDict = dict(zip(featureNames, columns))

pprint(featuresDict)

{'B11': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'B12': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'B2': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'B3': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'B4': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'B8': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'QA60': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'lulcid': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None)}


In [0]:
def parse_tfrecord(example_proto):
  parsed_features = tf.io.parse_single_example(example_proto, featuresDict)
  labels = parsed_features.pop(label)
  return parsed_features, tf.cast(labels, tf.int32)
parsedDataset = trainDataset.map(parse_tfrecord, num_parallel_calls=5)
pprint(iter(parsedDataset).next())

({'B11': <tf.Tensor: id=54, shape=(1,), dtype=float32, numpy=array([0.3501], dtype=float32)>,
  'B12': <tf.Tensor: id=55, shape=(1,), dtype=float32, numpy=array([0.1975], dtype=float32)>,
  'B2': <tf.Tensor: id=56, shape=(1,), dtype=float32, numpy=array([0.1373], dtype=float32)>,
  'B3': <tf.Tensor: id=57, shape=(1,), dtype=float32, numpy=array([0.1312], dtype=float32)>,
  'B4': <tf.Tensor: id=58, shape=(1,), dtype=float32, numpy=array([0.1545], dtype=float32)>,
  'B8': <tf.Tensor: id=59, shape=(1,), dtype=float32, numpy=array([0.2692], dtype=float32)>,
  'QA60': <tf.Tensor: id=60, shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>},
 <tf.Tensor: id=61, shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>)


In [0]:
def normalizedDifference(a, b):
  nd = (a - b) / (a + b)
  nd_inf = (a - b) / (a + b + 0.000001)
  return tf.where(tf.math.is_finite(nd), nd, nd_inf)

def addNDVI(features, label):
  features['NDVI'] = normalizedDifference(features['B8'], features['B4'])
  return features, label

## Creating simple neural network model (64 node hidden layer) 

In [0]:
from tensorflow import keras
nClasses = 7
inputDataset = parsedDataset.map(addNDVI)
def toTuple(dict, label):
  return tf.transpose(list(dict.values())), tf.one_hot(indices=label, depth=nClasses)
inputDataset = inputDataset.map(toTuple).repeat().batch(10)
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(64, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(nClasses, activation=tf.nn.softmax)
])
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x=inputDataset, epochs=10, steps_per_epoch=100)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/10
100/100 [==============================] - 7s 67ms/step - loss: 1.5987 - acc: 0.4702
Epoch 2/10
100/100 [==============================] - 5s 54ms/step - loss: 1.1241 - acc: 0.6910
Epoch 3/10
100/100 [==============================] - 6s 59ms/step - loss: 1.1332 - acc: 0.6231
Epoch 4/10
100/100 [==============================] - 5s 54ms/step - loss: 1.0472 - acc: 0.7292
Epoch 5/10
100/100 [==============================] - 5s 54ms/step - loss: 1.0491 - acc: 0.6857
Epoch 6/10
100/100 [==============================] - 6s 59ms/step - loss: 1.0938 - acc: 0.6336
Epoch 7/10
100/100 [==============================] - 5s 53ms/step - loss: 1.0259 - acc: 0.7250
Epoch 8/10
100/100 [==============================] - 5s 54ms/step - loss: 1.0365 - acc: 0.6797
Epoch 9/10
100/100 [==============================] - 6s 59ms/step - loss: 1.0655 - acc: 0.6447
Epoch 10/10
100/100 [=====================

## Testing model accuracy

In [0]:
testDataset = (
  tf.data.TFRecordDataset(testFilePath, compression_type='GZIP')
    .map(parse_tfrecord, num_parallel_calls=5)
    .map(addNDVI)
    .map(toTuple)
    .batch(1)
)

model.evaluate(testDataset, steps=45)

45/45 [==============================] - 1s 25ms/step - loss: 0.8467 - acc: 0.7556


[0.8467321998543209, 0.75555557]

## Classify image from EE using the trained model.

In [0]:
filesList = !gsutil ls 'gs://'{outputBucket}
exportFilesList = [s for s in filesList if imageFilePrefix in s]
imageFilesList = []
jsonFile = None
for f in exportFilesList:
  if f.endswith('.tfrecord.gz'):
    imageFilesList.append(f)
  elif f.endswith('.json'):
    jsonFile = f
imageFilesList.sort()
pprint(imageFilesList)
print(jsonFile)

['gs://gic-rohit12.appspot.com/Image_pixel_demo_00000.tfrecord.gz',
 'gs://gic-rohit12.appspot.com/Image_pixel_demo_00001.tfrecord.gz',
 'gs://gic-rohit12.appspot.com/Image_pixel_demo_00002.tfrecord.gz',
 'gs://gic-rohit12.appspot.com/Image_pixel_demo_00003.tfrecord.gz',
 'gs://gic-rohit12.appspot.com/Image_pixel_demo_00004.tfrecord.gz',
 'gs://gic-rohit12.appspot.com/Image_pixel_demo_00005.tfrecord.gz',
 'gs://gic-rohit12.appspot.com/Image_pixel_demo_00006.tfrecord.gz',
 'gs://gic-rohit12.appspot.com/Image_pixel_demo_00007.tfrecord.gz',
 'gs://gic-rohit12.appspot.com/Image_pixel_demo_00008.tfrecord.gz',
 'gs://gic-rohit12.appspot.com/Image_pixel_demo_00009.tfrecord.gz',
 'gs://gic-rohit12.appspot.com/Image_pixel_demo_00010.tfrecord.gz']
gs://gic-rohit12.appspot.com/Image_pixel_demo_mixer.json


In [0]:
import json
jsonText = !gsutil cat {jsonFile}
mixer = json.loads(jsonText.nlstr)
pprint(mixer)

{'patchDimensions': [256, 256],
 'patchesPerRow': 17,
 'projection': {'affine': {'doubleMatrix': [8.983152841195215e-05,
                                            0.0,
                                            104.78308800083747,
                                            0.0,
                                            -8.983152841195215e-05,
                                            15.314568795198014]},
                'crs': 'EPSG:4326'},
 'totalPatches': 170}


In [0]:
PATCH_WIDTH = mixer['patchDimensions'][0]
PATCH_HEIGHT = mixer['patchDimensions'][1]
PATCHES = mixer['totalPatches']
PATCH_DIMENSIONS_FLAT = [PATCH_WIDTH * PATCH_HEIGHT, 1]
imageColumns = [
  tf.FixedLenFeature(shape=PATCH_DIMENSIONS_FLAT, dtype=tf.float32) 
    for k in bands
]

imageFeaturesDict = dict(zip(bands, imageColumns))
imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')
def parse_image(example_proto):
  return tf.io.parse_single_example(example_proto, imageFeaturesDict)
imageDataset = imageDataset.map(parse_image, num_parallel_calls=7)
imageDataset = imageDataset.flat_map(
  lambda features: tf.data.Dataset.from_tensor_slices(features)
)
imageDataset = imageDataset.map(
  lambda features: addNDVI(features, None)[0]
)
imageDataset = imageDataset.map(
  lambda dataDict: (tf.transpose(list(dataDict.values())), tf.constant(-1))
)
imageDataset = imageDataset.batch(PATCH_WIDTH * PATCH_HEIGHT)

In [0]:
predictions = model.predict(imageDataset, steps=PATCHES, verbose=1)
print(predictions[0])

170/170 [==============================] - 1235s 7s/step
[[0.00152943 0.67150086 0.03643306 0.11765919 0.01541638 0.10929399
  0.04816714]]


In [0]:
outputImageFile = 'gs://' + outputBucket + '/Classified_pixel_demo.TFRecord'
print('Writing to file ' + outputImageFile)

Writing to file gs://gic-rohit12.appspot.com/Classified_pixel_demo.TFRecord


In [0]:
writer = tf.python_io.TFRecordWriter(outputImageFile)
patch = [[], [], [], [], [], [], [], []]
curPatch = 1
for prediction in predictions:
  patch[0].append(tf.argmax(prediction, 1))
  patch[1].append(prediction[0][0])
  patch[2].append(prediction[0][1])
  patch[3].append(prediction[0][2])
  patch[4].append(prediction[0][3])
  patch[5].append(prediction[0][4])
  patch[6].append(prediction[0][5])
  if (len(patch[0]) == PATCH_WIDTH * PATCH_HEIGHT):
    print('Done with patch ' + str(curPatch) + ' of ' + str(PATCHES) + '...')
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'prediction': tf.train.Feature(
              int64_list=tf.train.Int64List(
                  value=patch[0])),
          'agricultureProb': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[1])),
          'forestProb': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[2])),
          'marshProb': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[3])),
          'fieldsProb': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[4])),
          'urbanprob': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[5])),
          'waterProb': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[6])),
        }
      )
    )
    writer.write(example.SerializeToString())
    patch = [[], [], [], [], [], [], []]
    curPatch += 1

writer.close()

Done with patch 1 of 170...
Done with patch 2 of 170...
Done with patch 3 of 170...
Done with patch 4 of 170...
Done with patch 5 of 170...
Done with patch 6 of 170...
Done with patch 7 of 170...
Done with patch 8 of 170...
Done with patch 9 of 170...
Done with patch 10 of 170...
Done with patch 11 of 170...
Done with patch 12 of 170...
Done with patch 13 of 170...
Done with patch 14 of 170...
Done with patch 15 of 170...
Done with patch 16 of 170...
Done with patch 17 of 170...
Done with patch 18 of 170...
Done with patch 19 of 170...
Done with patch 20 of 170...
Done with patch 21 of 170...
Done with patch 22 of 170...
Done with patch 23 of 170...
Done with patch 24 of 170...
Done with patch 25 of 170...
Done with patch 26 of 170...
Done with patch 27 of 170...
Done with patch 28 of 170...
Done with patch 29 of 170...
Done with patch 30 of 170...
Done with patch 31 of 170...
Done with patch 32 of 170...
Done with patch 33 of 170...
Done with patch 34 of 170...
Done with patch 35 of 1

## Uploading classified image to Google Earth Engine Asset

In [0]:
!gsutil ls -l {outputImageFile}

 278565060  2020-01-31T06:30:59Z  gs://gic-rohit12.appspot.com/Classified_pixel_demo.TFRecord
TOTAL: 1 objects, 278565060 bytes (265.66 MiB)


In [0]:
USER_NAME = 'rhtkhati'
outputAssetID = 'users/' + USER_NAME + '/Classified_pixel_demo'
print('Writing to ' + outputAssetID)

Writing to users/rhtkhati/Classified_pixel_demo


In [0]:
!earthengine --no-use_cloud_api upload image --asset_id={outputAssetID} {outputImageFile} {jsonFile}

Started upload task with ID: 65MCTZLA4UR3WFVJNFGTAYOY


In [0]:
ee.batch.Task.list()

[<Task INGEST: Asset ingestion: users/rhtkhati/Classified_pixel_demo (READY)>,
 <Task EXPORT_IMAGE: Image Export (COMPLETED)>,
 <Task EXPORT_FEATURES: Testing Export (COMPLETED)>,
 <Task EXPORT_FEATURES: Training Export (COMPLETED)>,
 <Task EXPORT_IMAGE: Image Export (COMPLETED)>,
 <Task EXPORT_FEATURES: Testing Export (COMPLETED)>,
 <Task EXPORT_FEATURES: Training Export (COMPLETED)>,
 <Task EXPORT_IMAGE: Image Export (COMPLETED)>,
 <Task EXPORT_FEATURES: Testing Export (COMPLETED)>,
 <Task EXPORT_FEATURES: Training Export (COMPLETED)>]

## Visualizing ingested asset using Folium.

In [0]:
predictionsImage = ee.Image(outputAssetID)

predictionVis = {
  'bands': 'prediction',
  'min': 0,
  'max': 2,
  'palette': ['red', 'yellow', 'blue']
}
probabilityVis = {'bands': ['agricultureProb', 'forestProb', 'waterProb']}

predictionMapid = predictionsImage.getMapId(predictionVis)
probabilityMapid = predictionsImage.getMapId(probabilityVis)

map = folium.Map(location=[15.2448, 104.8473])
folium.TileLayer(
  tiles=predictionMapid['tile_fetcher'].url_format,
  attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
  overlay=True,
  name='prediction',
).add_to(map)
folium.TileLayer(
  tiles=probabilityMapid['tile_fetcher'].url_format,
  attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
  overlay=True,
  name='probability',
).add_to(map)
map.add_child(folium.LayerControl())
map